In [1]:
import os
from ensure import ensure_annotations

In [2]:
os.chdir("../")

In [3]:
!pwd

/Users/ngkuissi/Dev/Image_Search_Engine


In [4]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass(frozen=True)
class ModelEvaluationConfig:
    model_path: Path
    image_labels_path: Path
    image_path_list_dir: Path
    val_path: Path
    val_feature: Path
    n_neighbors: int
    return_distance: bool
    include_top: bool
    pooling: str
    input_shape: List
    target_size: List


In [5]:
from imageSearchEngine.constants import *
from imageSearchEngine.utils.file_helpers import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    @ensure_annotations
    def __init__(
        self,
        config_filepath:Path = CONFIG_FILE_PATH,
        params_filepath:Path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    @ensure_annotations
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.model_evaluation
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            model_path= config.model_path,
            image_labels_path= config.image_labels_path,
            image_path_list_dir = config.image_path_list_dir,
            val_feature= config.val_feature,
            val_path=config.val_path,
            n_neighbors=params.n_neighbors,
            return_distance= params.return_distance,
            include_top= params.include_top,
            pooling = params.pooling,
            input_shape= params.input_shape,
            target_size= params.target_size
        )

        return model_evaluation_config


In [7]:
import os
from tqdm import tqdm
import pickle
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow .keras.applications import resnet50
import numpy as np
from imageSearchEngine.logging.logger import log
from imageSearchEngine.exception import CustomException

In [8]:
class ModelEvaluation:

    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    @staticmethod
    def _precision(predicted, actual):
        prec = [value for value in predicted if value in actual]
        prec = float(len(prec)) / float(len(predicted))
        return prec
    
    @staticmethod
    def _apk(actual: list, predicted: list, k=10) -> float:
        """
        Computes the average precision at k.
        Parameters
        ----------
        actual : list
            A list of actual items to be predicted
        predicted : list
            An ordered list of predicted items
        k : int, default = 10
            Number of predictions to consider
        Returns:
        -------
        score : float
            The average precision at k.
        """
        if not predicted or not actual:
            return 0.0
        
        if len(predicted) > k:
            predicted = predicted[:k]

        score = 0.0
        true_positives = 0.0

        for i, p in enumerate(predicted):
            if p in actual and p not in predicted[:i]:
                max_ix = min(i + 1, len(predicted))
                score += ModelEvaluation._precision(predicted[:max_ix], actual)
                true_positives += 1
        
        if score == 0.0:
            return 0.0
        
        return score / true_positives
    
    @staticmethod
    def mapk(actual: List[list], predicted: List[list], k: int=10) -> float:
        """
        Computes the mean average precision at k.
        Parameters
        ----------
        actual : a list of lists
            Actual items to be predicted
            example: [['A', 'B', 'X'], ['A', 'B', 'Y']]
        predicted : a list of lists
            Ordered predictions
            example: [['X', 'Y', 'Z'], ['X', 'Y', 'Z']]
        Returns:
        -------
            mark: float
                The mean average precision at k (map@k)
        """
        if len(actual) != len(predicted):
            raise AssertionError("Length mismatched")
        
        return np.mean([ModelEvaluation._apk(a,p,k) for a,p in zip(actual, predicted)])

    def create_val_features(self):
        val_features = []
        val_image_path = []
        for label in os.listdir(self.config.val_path):
            image_path = os.path.join(self.config.val_path, f'{label}/images')
            for image in os.listdir(image_path):
                val_image_path.append(os.path.join(image_path, image))
        log.info('all the val_path were retrived')

        model = resnet50.ResNet50(
            include_top=self.config.include_top,
            input_shape=self.config.input_shape,
            pooling=self.config.pooling
        )
        log.info('got resnet50 model from tensorflow')
        log.info("new logs")
        for image_path in tqdm(val_image_path):
            image = load_img(image_path, target_size=(224, 224))
            image = img_to_array(image)
            image = np.expand_dims(image, axis=0)
            preprocess_image = resnet50.preprocess_input(image)
            feature = model.predict(preprocess_image, verbose=0).flatten()
            # getting label for the image
            label = image_path.split('/')[-3]
            log.info(label)
            val_features.append((feature, label))
        
        return None
        log.info('All the validation data was loaded')
        pickle.dump(val_features, open(self.config.val_feature, 'wb'))
        log.info('All the validation data was saved to thier repsective path')
    
    def get_predictions(self):
        model = pickle.load(open(self.config.model_path, 'rb'))
        predictions = []
        test_labels = []
        #make sure we have the features
        if not os.path.exists(self.config.val_feature):
            log.info("the path for the label feature does not exist creating them now")
            self.create_val_features()
        
        test_feature = pickle.load(open(self.config.val_feature, 'rb'))
        predictions = []
        test_labels = []
        labels = pickle.load(open(self.config.image_labels_path, 'rb'))
        image_path_list = pickle.load(open(self.config.image_path_list_dir, 'rb'))
        model: KNeighborsClassifier = pickle.load(open(self.config.model_path, 'rb'))

        for feature_x, label in tqdm(test_feature):
            index_list = model.kneighbors(feature_x.reshape(1, -1),
                                          n_neighbors=self.config.n_neighbors, return_distance=self.config.return_distance
                                          )
            print(index_list)
            prediction_name = [image_path_list[i].split('/')[-1] for i in  index_list]
            print(prediction_name)
            predictions.append(prediction_name)
            
            test_labels.append(labels[label])
        log.info("all the predictions were made for the test dataset")
        
        return predictions, test_labels

    def evaluate_model(self):
        predictions, test_labels = self.get_predictions()
        score = self.mapk(test_labels, predicted=predictions, k=self.config.n_neighbors)
        log.info(f"the score is for the model is {score}")
        
        
        




In [9]:
try:
    configuration = ConfigurationManager()
    model_evaluation_config =  configuration.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate_model()
    #model_evaluation.create_val_features()
except Exception as e:
    raise CustomException(e)

[2023-10-06 18:01:10,719: INFO: file_helpers: yaml file: config/config.yaml loaded successfully]
[2023-10-06 18:01:10,720: INFO: file_helpers: yaml file: params.yaml loaded successfully]
[2023-10-06 18:01:10,721: INFO: file_helpers: created directory at: artifacts]
[2023-10-06 18:01:10,721: INFO: file_helpers: created directory at: artifacts/model_evaluation]


  0%|          | 0/10000 [00:00<?, ?it/s]

(array([[33.40074462, 33.72742119, 33.87100162, 33.96783901, 34.33271014,
        34.56586748, 34.7527996 , 35.24854378, 35.44515666, 35.50906299,
        35.56848073, 35.99504564, 36.00712176, 36.15936778, 36.31463123,
        36.43143074, 36.43799917, 36.5012558 , 36.52662837, 36.53467992]]), array([[90572, 56957,    56, 96211, 19422, 91277,   479, 56597,  2767,
        90814, 13684, 43025, 56854, 13714, 48222, 62502, 41531, 70374,
           43, 68230]]))


[2023-10-06 18:01:11,684: INFO: exception: Error occured in python script name [/var/folders/7r/vsv5qx453z3931l81y_t76y80000gn/T/ipykernel_58888/184301285.py] line number [5] error message[only integer scalar arrays can be converted to a scalar index]]
[2023-10-06 18:01:11,694: INFO: exception: Error occured in python script name [/var/folders/7r/vsv5qx453z3931l81y_t76y80000gn/T/ipykernel_58888/184301285.py] line number [5] error message[only integer scalar arrays can be converted to a scalar index]]


CustomException: Error occured in python script name [/var/folders/7r/vsv5qx453z3931l81y_t76y80000gn/T/ipykernel_58888/184301285.py] line number [5] error message[only integer scalar arrays can be converted to a scalar index]

[2023-10-06 18:01:11,695: INFO: exception: Error occured in python script name [/var/folders/7r/vsv5qx453z3931l81y_t76y80000gn/T/ipykernel_58888/184301285.py] line number [5] error message[only integer scalar arrays can be converted to a scalar index]]


In [ ]:
!pwd

/Users/ngkuissi/Dev/Image_Search_Engine
